<img src="img/Marca-ITBA-Color-ALTA.png" width="250">

# Programación para el Análisis de Datos

## Clase 5 - Estadística descriptiva - Análisis exploratorio de los datos (EDA)


## Introducción

Cualquier proyecto de análisis de datos comienza por una etapa de limpieza y exploración de los datos usando herramientas de la estadística descriptiva. Este trabajo es previo a la implementación de cualquier tipo de modelado y casi siempre nos da información relevante sobre los datos por sí mismo. El objetivo es caracterizar qué estructura presentan los datos. Para ello debemos identificar qué tipo de variables componen al dataset (categóricas, numéricas), ver las distribuciones e identificar anomalías como la presencia de valores extremos, ver si hay un desbalance de clases para variables categóricas y tratar de identificar cómo interactúan entre sí las diferentes variables. Conocer estos aspectos de los datos nos servirá para plantear nuevas hipótesis y será fundamental como paso previo a elegir qué modelo implementar en cada problema y hacer las transformaciones pertinentes para hacer una interpretación correcta de los resultados. 

En esta notebook trabajaremos con un dataset de métricas relativas a posteos publicitarios en facebook. El mismo se refiere a posteos hechos durante el 2014 por una reconocida marca de consméticos. Entre las variables veremos atributos conocidos previamente al posteo y métricas de impacto del mismo. Nuestro objetivo será encontrar qué relación existe entre estos atributos y las variables de impacto. 

Los atributos son

* **Category:** caracterización manual del contenido en tres clases. 1. Acción (ofertas especiales, concursos, etc); 2. Producto (publicidad directa sobre un producto, contenido explícito de la marca) ; 3. Inspiración (contenido no relacionado explícitamente con la marca).

* **Page total likes:** número de personas que likearon la página de la compañía al momento del posteo.

* **Mes del posteo**

* **Día de la semana del posteo**

* **Hora del posteo**

* **Pago:** si la compañía le pagó a facebook para publicitar el posteo

Las métricas de impacto son:

* **Lifetime post total reach:** el número de personas que vieron el post (cada persona se cuenta una vez, aunque lo haya visto varias veces)

* **Lifetime post total impressions:** el número de veces que fue presentado el post, puede ser que una misma persona vea más de una vez el post por distintas vias de llegada.

* **Lifetime engaged users:** número de personas que clickearon en algún lugar del post (ver imagen abajo)

* **Lifetime post consumers:** número de personas que clickearon en algún lugar del post (ver imagen abajo)

* **Lifetime post consumptions:** número de clicks sobre el post

* **Lifetime post impressions by people who have liked a page:** el número de impresiones debidas a personas que habían likeado la página.

* **Lifetime post reach by people who like a page:** número de personas que vieron el post porque habían likeado la página (cada persona se cuenta una sola vez).

* **Lifetime people who have liked a page and engaged with a post**

* **Comments:** número de comentarios en el post.

* **Likes:** número de likes en el post.

* **Shares:** número de veces que el post fue compartido.

* **Total interactions**: la suma de comentarios, likes y shares.


<img src="img/interactions_fb.png">




### Referencia

S. Moro, P. Rita and B. Vala. Predicting social media performance metrics and evaluation 
of the impact on brand building: A data mining approach. Journal of Business Research, Elsevier, In press.

Disponible en el siguiente <a href="http://dx.doi.org/10.1016/j.jbusres.2016.02.010">link</a>

El dataset se encuentra disponible en el siguiente <a href="https://archive.ics.uci.edu/ml/datasets/Facebook+metrics">link</a>



In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.float_format',  '{:,.2f}'.format)

In [2]:
facebook = pd.read_csv('data/dataset_Facebook.csv', sep=';')

In [ ]:
facebook.shape

In [ ]:
facebook.head()

### Limpieza

Si bien este dataset ya está bastante limpio vamos a renombrar por comodidad algunas variables y remover valores nulos.

In [5]:
# Renombramos columnas
new_names = {
           'Post Month':'Month',
           'Post Weekday':'Weekday',
           'Post Hour':'Hour',
           'Lifetime Post Total Reach':'Reach',
           'Lifetime Post Total Impressions':'Impressions',
           'Lifetime Engaged Users':'Engaged users',
           'Lifetime Post Consumers':'Consumers',
           'Lifetime Post Consumptions':'Consumptions',
           'Lifetime Post Impressions by people who have liked your Page':'Impressions by followers',
           'Lifetime Post reach by people who like your Page':'Reach by followers',
           'Lifetime People who have liked your Page and engaged with your post':'Engaged followers',    
           'comment':'Comments',
           'like':'Likes',
           'share':'Shares',
           'Total Interactions':'Total interactions'    
           }
facebook.rename(columns=new_names, inplace=True)

In [ ]:
facebook.info()

#### Valores faltantes

Vemos que hay muy pocos valores nulos y por el momento vamos a dropearlos. Luego de hacer el análisis exploratorio podríamos definir algún criterio de imputación. Por ejemplo deberíamos ver si conviene usar la media o la mediana y si la imputación la haremos mirando la distribución total de cada variable o condicionada a ciertas categorías.

In [7]:
# Dropeamos los nulos
facebook.dropna(inplace=True)

In [8]:
# Identificamos variables categoricas y numericas
categorical_vars = ['Type', 'Category', 'Month', 'Weekday', 'Paid']
numeric_vars = ['Page total likes', 'Comments','Consumers','Consumptions','Engaged followers','Engaged users','Impressions','Impressions by followers','Likes','Reach','Reach by followers','Shares','Total interactions']


### Análisis exploratorio de los datos (EDA)

#### Métricas de posición y dispersión

In [ ]:
description = facebook[numeric_vars].describe().transpose()
description['cv'] = description['std'] / description['mean']
description = description[['mean','std','cv','min','25%','50%','75%','max']]
description

#### Histogramas

Si bien estos estadísticos nos dan un primer pantallazo sobre la estructura de los datos, lo hacen de manera demasiado resumida. Veamos gráficamente cómo se distribuyen las variables.

In [ ]:
# Distribuciones para cada variable numerica
f, ax = plt.subplots(3,5, figsize=(20,14), gridspec_kw={'wspace':0.5,'hspace':0.3})

ax = ax.ravel()

for i, col in enumerate(numeric_vars):
    sns.histplot(facebook[col].astype(float), ax=ax[i], kde=False)
    ax[i].axvline(x=facebook[col].mean(), color='k', label='mean')
    ax[i].axvline(x=facebook[col].median(), color='r', label='median')    
ax[0].legend();
ax[-2].axis('off')
ax[-1].axis('off');

Vemos que muchos histogramas prácticamente no se ven porque la presencia de valores extremos distorsiona la escala. Volvemos a graficar los histogramas dentro del rango que nos parece razonable en cada caso.

In [ ]:
scales={'Comments':[0,80],
       'Consumers':[0,5000],
       'Consumptions':[0,10000],
       'Engaged followers':[0,3000],
       'Engaged users':[0,5000],
       'Impressions':[0,100000],
       'Impressions by followers':[0,100000],
       'Likes':[0,1000],
       'Page total likes':[75000,145000],
       'Reach':[0,50000],
       'Reach by followers':[0,40000],
       'Shares':[0,200],
       'Total interactions':[0,1000]      
      }

f, ax = plt.subplots(3,5, figsize=(20,14), gridspec_kw={'wspace':0.5,'hspace':0.3})

ax=ax.ravel()

for i, col in enumerate(numeric_vars):
    sns.histplot(facebook[col].astype(float), ax=ax[i], kde=False)
    ax[i].axvline(x=facebook[col].mean(), color='k', label='mean')
    ax[i].axvline(x=facebook[col].median(), color='r', label='median')
    ax[i].axis(xmin=scales[col][0], xmax=scales[col][1])
    
ax[0].legend();
ax[-2].axis('off')
ax[-1].axis('off');

Se ve que más allá de la presencia de outliers, las distribuciones son asimétricas con colas pesadas. Esto explica la diferencia entre la media y la mediana que se ve en las lineas rojas y negras en cada caso.

Hacemos gráficos de barra para ver cómo se distribuyen las variables categóricas.

In [ ]:
f, ax = plt.subplots(2,3, figsize=(20,12))
ax = ax.ravel()
for i, col in enumerate(categorical_vars):
    sns.countplot(x=col, data=facebook, ax=ax[i], palette='viridis', hue=col, legend=False);

ax[-1].axis('off');

### Relación entre variables

Los scatterplots nos permiten ver relaciones entre variables en la data cruda. En problemas multidimensionales puede ser impracticable hacer un pairplot para ver todos los pares de variables relacionados. Sin embargo es una buena práctica elegir al menos un subgrupo de variables de interés para explorarlos visualmente.

In [ ]:
# Relación entre variables de impacto
impact_vars = ['Reach',\
               'Impressions',\
               'Engaged users',\
               'Consumers',\
               'Consumptions',\
               'Total interactions']

sns.pairplot(data=facebook, vars=impact_vars);

In [ ]:
facebook[impact_vars].corr()

In [ ]:
# CORRELACIONES ENTRE VARIABLES DE IMPACTO
plt.figure(figsize=(15,10))
sns.heatmap(facebook[impact_vars].corr(), annot=True, cmap='RdBu', vmin=-1);

Vemos que hay nucleos de variables fuertemente correlacionadas. Siempre conviene complementar los coeficientes de correlación con una inspección de los scatterplots como la hecha más arriba. De otra manera podríamos perdernos de identificar relaciones no lineales entre las variables.

In [ ]:
# RELACION ENTRE ATRIBUTOS DEL POST Y VARIABLES DE IMPACTO (Total interactions,'Consumptions')
sns.pairplot(data=facebook,\
             x_vars=categorical_vars,\
             y_vars=['Total interactions','Consumptions'],\
             plot_kws={'alpha': 0.3});


In [ ]:
# Relevancia de las variables temporales

plt.figure(figsize=(10,6))
sns.boxplot(x='Month',y='Total interactions',data=facebook)
plt.ylim(0,750)
plt.ylabel("Total interactions")

plt.figure(figsize=(10,6))
sns.boxplot(x='Weekday',y='Total interactions',data=facebook)
plt.ylim(0,750)
plt.ylabel("Total interactions")

plt.figure(figsize=(10,6))
sns.boxplot(x='Hour',y='Total interactions',data=facebook)
plt.ylim(0,750)
plt.ylabel("Total interactions");


In [ ]:
# Importancia de pagar por publicidad
sns.countplot(x='Paid', data=facebook);

In [ ]:
sns.boxplot(data=facebook, x='Paid', y='Total interactions')
plt.ylim([0,750]);

* Vemos que en términos de la mediana, los post pagos obtienen más interacciones

In [ ]:
facebook['Paid'] = facebook['Paid'].astype(str)
sns.countplot(x='Weekday', hue='Paid', data=facebook);

* La empresa invierte más en publicidad los días medios de la semana

In [ ]:
# Relacion entre mes y pago
sns.countplot(x='Month', hue='Paid', data=facebook);

In [ ]:
# Interacción entre hora y pago
sns.countplot(x='Hour', hue='Paid', data=facebook);

In [ ]:
# Vemos que la compañía tiende a pagar más por los anuncios en mitad de la semana.
# Veamos cómo se refleja esto en el impacto de los posteos.

plt.figure(figsize=(10,6))
sns.boxplot(x='Weekday', y='Total interactions', hue='Paid', data=facebook, palette='viridis')
plt.ylim(0,750)
plt.ylabel("Total interactions");


In [ ]:
# Interacción entre tipo de post y pago
sns.countplot(x='Type', hue='Paid', data=facebook);

In [ ]:
# Interacción entre tipo y categoria
# 1. Acción (ofertas especiales, concursos, etc); 
# 2. Producto (publicidad directa sobre un producto, contenido explícito de la marca) 
# 3. Inspiración (contenido no relacionado explícitamente con la marca).

sns.countplot(x='Type', hue='Category', data=facebook);

In [ ]:
# Importancia de la categoría en el impacto
sns.boxplot(x='Category', y='Total interactions', hue='Paid', data=facebook);
plt.ylim([0,1000]);

In [ ]:
# Importancia del tipo de post en el impacto
sns.boxplot(x='Type', y='Total interactions', hue='Paid', data=facebook);
plt.ylim([0,1000]);

- En promedio los post de tipo video obtienen más interacciones
- Los posteos de fotos tienen el rango de variabilidad más grande, esto sugiere que el impacto depende fuertemente del contenido de la foto.
- Los links tiene la peor performance en promedio.
- No se ve diferencia entre pagar y no pagar por posteos de tipo link

### ¡Muchas gracias por la atención!